### $\delta = t_2 - t_1$ vs. MMR

In [1]:
import pandas as pd
import numpy as np
from numpy import cov
from scipy.stats import pearsonr
import datetime
t = datetime.datetime.now()
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
folder = r'C:\Users\Ensheng\Desktop\Coronavirus\COVID_MMR\data\\'
output_folder =r'C:\Users\Ensheng\Desktop\Coronavirus\COVID_MMR\output\\'

In [2]:
# pre-config
logMMR = 0
cases_or_deaths = 'cases' # 'cases' 'deaths' 'ratio'
ex_list = ['Alaska','American Samoa','Guam','Northern Mariana Islands',
           'Puerto Rico','Virgin Islands','Diamond Princess','Grand Princess']

#-----

# case-fatality ratio
# inc_value1 = 0
# inc_value2 = 0.05

# mortality rate
# inc_value1 = 2
# inc_value2 = 40

# incidence rate
# inc_value1 = 50
# inc_value2 = 1500

#-----

# case-fatality ratio
#case_value1 = 0

# deaths
#case_value1 = 1

# cases
#case_value1 = 10

In [3]:
# data: MMR at the county level
in_table = folder + r'US_MMR.xlsx'
mmr_df = pd.read_excel(in_table,sheet_name='county')
print(len(mmr_df))
mmr_df.head(3)

3102


,FIPS,County,State,MMR,Population,logMMR
0,1001,Autauga,AL,0.9536,55504,-0.020634
1,1003,Baldwin,AL,0.9700,212628,-0.013228
2,1005,Barbour,AL,0.9283,25270,-0.032312


In [4]:
# remove state with unified MMR
state_list = list(set(mmr_df['State']))
# detect states with the same MMR rate for all counties
one_vr_list = []
for state in state_list:
    if (len(set(mmr_df[mmr_df['State']==state]['MMR']))) == 1:
        print(state)
        one_vr_list.append(state)
mmr_df =  mmr_df[~mmr_df['State'].isin(one_vr_list)]
print(len(mmr_df))

NV
NE
OH
AR
NH
MS
WV
HI
DE
2674


In [5]:
# if logMMR instead of MMR
#logMMR = 0
if logMMR == 1:
    mmr_df['MMR'] = mmr_df['logMMR']
mmr_df = mmr_df.drop(columns=['logMMR'])
print(len(mmr_df))
mmr_df.head(3)

2674


,FIPS,County,State,MMR,Population
0,1001,Autauga,AL,0.9536,55504
1,1003,Baldwin,AL,0.9700,212628
2,1005,Barbour,AL,0.9283,25270


In [6]:
if cases_or_deaths == 'ratio':
    
    # calculate case-fatality ratio
    in_table = r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
    cases_df = pd.read_csv(in_table)
    cases_df = cases_df.drop(columns=['UID', 'iso2','iso3','code3','Country_Region','Lat','Long_','Combined_Key'])
    #cases_df = cases_df[cases_df[cases_df.columns[-1]] > 0] # only keep counties with cases/deaths
    cases_df = cases_df[~cases_df['Province_State'].isin(ex_list)]
    cases_df = cases_df[~cases_df['Admin2'].isin(['Unassigned'])]
    cases_df = cases_df[~cases_df['Admin2'].str.startswith('Out of')]
    cases_df = cases_df[cases_df['FIPS'].notnull()]
    cases_df.loc[cases_df['FIPS'].notnull(), 'FIPS'] = cases_df[cases_df['FIPS'].notnull()]['FIPS'].astype(int) # FIPS code from float to int
    print(cases_or_deaths + "! Number of counties with FIPS: " + str(len(cases_df)))
    c_df = cases_df
    c_df.tail(3)

    in_table = r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
    cases_df = pd.read_csv(in_table)
    cases_df = cases_df.drop(columns=['UID', 'iso2','iso3','code3','Country_Region','Lat','Long_','Combined_Key','Population'])
    #cases_df = cases_df[cases_df[cases_df.columns[-1]] > 0] # only keep counties with cases/deaths
    cases_df = cases_df[~cases_df['Province_State'].isin(ex_list)]
    cases_df = cases_df[~cases_df['Admin2'].isin(['Unassigned'])]
    cases_df = cases_df[~cases_df['Admin2'].str.startswith('Out of')]
    cases_df = cases_df[cases_df['FIPS'].notnull()]
    cases_df.loc[cases_df['FIPS'].notnull(), 'FIPS'] = cases_df[cases_df['FIPS'].notnull()]['FIPS'].astype(int) # FIPS code from float to int
    print(cases_or_deaths + "! Number of counties with FIPS: " + str(len(cases_df)))
    d_df = cases_df
    d_df.tail(3)

    print(list(c_df['FIPS']) == list(d_df['FIPS']))
    print(list(c_df.columns) == list(d_df.columns))

    d_df.set_index(['FIPS','Admin2','Province_State'], inplace=True)
    c_df.set_index(['FIPS','Admin2','Province_State'], inplace=True)
    ratio_df = d_df.div(c_df).reset_index()
    #ratio_df.dropna(axis='columns',how='all')
    ratio_df.tail()

In [7]:
#cases_or_deaths = 'ratio'

# data: cases at the county level
in_table = r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
cases_df = pd.read_csv(in_table)
cases_df = cases_df.drop(columns=['UID', 'iso2','iso3','code3','Country_Region','Lat','Long_','Combined_Key'])

# data: deaths at the county level
in_table = r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
deaths_df = pd.read_csv(in_table)
deaths_df = deaths_df.drop(columns=['UID', 'iso2','iso3','code3','Country_Region','Lat','Long_','Combined_Key','Population'])

if cases_or_deaths == 'deaths':
    cases_df = deaths_df
elif cases_or_deaths == 'ratio':
    cases_df = ratio_df
    
# clean the table
#ex_list = ['Alaska','American Samoa','Guam','Northern Mariana Islands','Puerto Rico','Virgin Islands','Diamond Princess','Grand Princess']
cases_df = cases_df[cases_df[cases_df.columns[-1]] > 0] # only keep counties with cases/deaths
cases_df = cases_df[~cases_df['Province_State'].isin(ex_list)]
cases_df = cases_df[~cases_df['Admin2'].isin(['Unassigned'])]
cases_df = cases_df[~cases_df['Admin2'].str.startswith('Out of')]
cases_df = cases_df[cases_df['FIPS'].notnull()]
cases_df.loc[cases_df['FIPS'].notnull(), 'FIPS'] = cases_df[cases_df['FIPS'].notnull()]['FIPS'].astype(int) # FIPS code from float to int
print(cases_or_deaths + "! Number of counties with FIPS: " + str(len(cases_df)))
cases_df.tail(3)

cases! Number of counties with FIPS: 3067


,FIPS,Admin2,Province_State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20
3336,56041,Uinta,Wyoming,0,0,0,0,0,0,0,...,278,277,279,278,278,277,276,276,277,278
3338,56043,Washakie,Wyoming,0,0,0,0,0,0,0,...,77,77,79,82,86,96,97,97,100,102
3339,56045,Weston,Wyoming,0,0,0,0,0,0,0,...,5,6,5,5,5,5,7,7,8,8


In [8]:
# counties with MMR but no cases
mmr_df[mmr_df['FIPS'].isin(set(mmr_df['FIPS']) - set(cases_df['FIPS']))]

,FIPS,County,State,MMR,Population
245,8061,Kiowa,CO,0.977064,1376
550,16061,Lewis,ID,0.917500,3887
933,20153,Rawlins,KS,1.000000,2497
956,20199,Wallace,KS,0.670000,1524
1189,25007,Dukes,MA,0.893000,17325
1195,25019,Nantucket,MA,0.967000,11229
1571,30011,Carter,MT,0.923000,1222
1600,30069,Petroleum,MT,1.000000,523
1769,35011,De Baca,NM,0.937000,1829
1798,36005,Bronx,NY,0.990000,1471160


In [9]:
# counties without MMR
cases_df[cases_df['FIPS'].isin(set(cases_df['FIPS']) - set(mmr_df['FIPS']))]

,FIPS,Admin2,Province_State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20
118,5001,Arkansas,Arkansas,0,0,0,0,0,0,0,...,217,221,227,233,241,243,246,247,249,251
119,5003,Ashley,Arkansas,0,0,0,0,0,0,0,...,326,330,330,335,337,342,352,362,363,362
120,5005,Baxter,Arkansas,0,0,0,0,0,0,0,...,71,71,73,76,78,81,81,86,89,89
121,5007,Benton,Arkansas,0,0,0,0,0,0,0,...,4774,4794,4805,4825,4864,4879,4900,4917,4933,4950
122,5009,Boone,Arkansas,0,0,0,0,0,0,0,...,211,212,214,217,222,223,225,230,230,232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3236,54101,Webster,West Virginia,0,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,6
3237,54103,Wetzel,West Virginia,0,0,0,0,0,0,0,...,42,43,44,44,44,44,45,45,45,45
3238,54105,Wirt,West Virginia,0,0,0,0,0,0,0,...,6,7,7,7,7,7,7,7,7,7
3239,54107,Wood,West Virginia,0,0,0,0,0,0,0,...,247,250,256,264,266,269,275,283,283,285


In [10]:
cases_df = pd.merge(mmr_df,cases_df, how='left', on='FIPS')
cases_df = cases_df.drop(columns=['Admin2','State'])
print(len(cases_df))
cases_df.head(3)

2674


,FIPS,County,MMR,Population,Province_State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20
0,1001,Autauga,0.9536,55504,Alabama,0.0,0.0,0.0,0.0,0.0,...,1169.0,1174.0,1162.0,1188.0,1196.0,1196.0,1206.0,1218.0,1219.0,1235.0
1,1003,Baldwin,0.9700,212628,Alabama,0.0,0.0,0.0,0.0,0.0,...,3544.0,3651.0,3670.0,3710.0,3744.0,3791.0,3839.0,3877.0,3888.0,3906.0
2,1005,Barbour,0.9283,25270,Alabama,0.0,0.0,0.0,0.0,0.0,...,575.0,576.0,579.0,581.0,586.0,590.0,598.0,604.0,605.0,606.0


---
### Section 1: By policy time

$\delta_c = c_{t2} - c_{t1}$

* $\delta_c$ is the cases between two policy time points.
* $c_t$ is the cases at time point 1 or 2

In [211]:
# import policy time
in_table = folder + r'policy_time_US.xlsx'
policy_df = pd.read_excel(in_table,sheet_name='NYT')
# define time points
time1 = 'public_schools' # 'stay_at_home_date' # 'public_schools'
time2 = 'SHO_expired_p1'
policy_df = policy_df[['Province_State',time1,time2]]
# remove nulls
policy_df = policy_df[policy_df[time1].notnull()]
policy_df = policy_df[policy_df[time2].notnull()]
# date format update
policy_df[time1] = pd.to_datetime(policy_df[time1]).dt.strftime('%#m/%#d/%y')
policy_df[time2] = pd.to_datetime(policy_df[time2]).dt.strftime('%#m/%#d/%y')
# remove rows
policy_df = policy_df[~policy_df['Province_State'].isin(ex_list)]
print(len(policy_df))
policy_df.head(3)

43


,Province_State,public_schools,SHO_expired_p1
0,Alabama,3/26/20,4/30/20
2,Arizona,3/16/20,5/15/20
4,California,3/19/20,5/8/20


In [212]:
state_list = list(set(policy_df['Province_State']))
state_list.sort(reverse=False)

In [213]:
# states without t1 or t2
set(cases_df['Province_State']) - set(state_list)

{'North Dakota', 'South Dakota', 'Utah', 'Vermont', 'Wyoming', nan}

In [214]:
# states without MMR
set(state_list) - set(cases_df['Province_State'])

{'Delaware',
 'District of Columbia',
 'Hawaii',
 'Mississippi',
 'Nevada',
 'New Hampshire',
 'Ohio',
 'West Virginia'}

In [215]:
state_df = pd.DataFrame()
for state in state_list:
    time_point1 = policy_df[policy_df['Province_State']==state][time1].iloc[0]
    time_point2 = policy_df[policy_df['Province_State']==state][time2].iloc[0]
    state_df_temp = cases_df[cases_df['Province_State']==state]
    state_df_temp.insert(5, 't1_date', time_point1)
    state_df_temp.insert(6, 't2_date', time_point2)
    state_df_temp = state_df_temp[['FIPS','County','Province_State','MMR','Population','t1_date','t2_date',time_point1,time_point2]]
    state_df_temp = state_df_temp.rename(columns={time_point1: "t1",time_point2: "t2"})
    state_df = state_df.append(state_df_temp, ignore_index=True)
print(str(len(state_df)) + ' counties, ' + str(len(set(state_df['Province_State']))) + ' states.')
state_df.head(3)

2472 counties, 35 states.


,FIPS,County,Province_State,MMR,Population,t1_date,t2_date,t1,t2
0,1001,Autauga,Alabama,0.9536,55504,3/26/20,4/30/20,6.0,44.0
1,1003,Baldwin,Alabama,0.9700,212628,3/26/20,4/30/20,5.0,174.0
2,1005,Barbour,Alabama,0.9283,25270,3/26/20,4/30/20,0.0,39.0


In [216]:
# calculate delta_c
state_df.loc[state_df['t1']==0, 't1'] = 1
state_df['delta_c'] = (state_df['t2'] - state_df['t1']).div(state_df['t1'])
state_df = state_df[state_df['delta_c'] > 0]
print(len(state_df))
state_df.head()

2178


,FIPS,County,Province_State,MMR,Population,t1_date,t2_date,t1,t2,delta_c
0,1001,Autauga,Alabama,0.9536,55504,3/26/20,4/30/20,6.0,44.0,6.333333
1,1003,Baldwin,Alabama,0.9700,212628,3/26/20,4/30/20,5.0,174.0,33.800000
2,1005,Barbour,Alabama,0.9283,25270,3/26/20,4/30/20,1.0,39.0,38.000000
3,1007,Bibb,Alabama,0.9432,22668,3/26/20,4/30/20,1.0,42.0,41.000000
4,1009,Blount,Alabama,0.9740,58013,3/26/20,4/30/20,2.0,37.0,17.500000


In [217]:
# calcuate the duration between stay_at_home_date and SHO_expired_p1
# ref: https://stackoverflow.com/questions/22132525/add-column-with-number-of-days-between-dates-in-dataframe-pandas
df = policy_df
df[time1] = pd.to_datetime(df[time1])
df[time2] = pd.to_datetime(df[time2])
df['duration'] = (df[time2] - df[time1]).dt.days
df.head()

,Province_State,public_schools,SHO_expired_p1,duration
0,Alabama,2020-03-26,2020-04-30,35
2,Arizona,2020-03-16,2020-05-15,60
4,California,2020-03-19,2020-05-08,50
5,Colorado,2020-03-18,2020-04-27,40
6,Connecticut,2020-03-17,2020-05-31,75


In [218]:
fig = px.histogram(df, x="duration", width=600, height=400)
fig.show()

In [219]:
# plot it

df = state_df

fig1 = px.scatter(df, x="MMR", y="delta_c", trendline="ols")
fig1.update_traces(textposition='top center')
fig1.update_layout(
        title= 'delta_c vs. MMR',
        xaxis=dict(
            title='MMR',
            gridcolor='white',
            gridwidth=2,
        ),
        yaxis=dict(
            title="Cases",
            gridcolor='white',
            gridwidth=2,
        ),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)', width=600, height=400
    )
fig1.show()

In [220]:
# from numpy import cov
# from scipy.stats import pearsonr

df = state_df
corr_df = pd.DataFrame()

for state in state_list:
    
    df_state = df[(df['Province_State']==state)]
    
    if len(df_state) >= 4:
    
        # covariance covariance[0,1],
        data1 = df_state['MMR']
        data2 = df_state['delta_c']
        covariance = cov(data1, data2)

        # Pearsons correlation
        corr, _ = pearsonr(data1, data2)

        # R-squared
        correlation_matrix = np.corrcoef(data1, data2)
        correlation_xy = correlation_matrix[0,1]
        r_squared = correlation_xy**2

        # outputs
        corr_df = corr_df.append([[state,str(len(df_state)),str(len(cases_df[cases_df['Province_State'] == state])),corr,'%.5f' % r_squared]])

corr_df.columns=['state','tested_counties', 'total_counties','pearsons','r_squared']
corr_df.sort_values(by='pearsons', ascending=True).reset_index(drop=True)

,state,tested_counties,total_counties,pearsons,r_squared
0,Florida,66,67,-0.306117,0.09371
1,Arizona,15,15,-0.289447,0.08378
2,South Carolina,46,46,-0.243718,0.05940
3,North Carolina,99,100,-0.236305,0.05584
4,Alabama,67,67,-0.178455,0.03185
5,Louisiana,64,64,-0.099991,0.01000
6,Tennessee,92,95,-0.087673,0.00769
7,Oregon,29,35,-0.071688,0.00514
8,Oklahoma,61,77,-0.051612,0.00266
9,Illinois,90,102,-0.049149,0.00242


In [221]:
corr_df = pd.DataFrame()

for state in state_list:
    
    df_state = state_df[(state_df['Province_State']==state)]
    
    if len(df_state) >= 4:
    
        # covariance
        data1 = df_state['MMR']
        data2 = df_state['delta_c']
        covariance = cov(data1, data2)

        # Pearsons correlation
        corr, _ = pearsonr(data1, data2)

        # R-squared
        correlation_matrix = np.corrcoef(data1, data2)
        correlation_xy = correlation_matrix[0,1]
        r_squared = correlation_xy**2

        # outputs
        corr_df = corr_df.append([[state,covariance[0,1],corr,r_squared]])

corr_df.columns=['State','Covariance','Pearsons_correlation','R_squared']
corr_df.sort_values(by='Pearsons_correlation', ascending=True)

,State,Covariance,Pearsons_correlation,R_squared
0,Florida,-3.909559,-0.306117,0.093707
0,Arizona,-4.434676,-0.289447,0.083780
0,South Carolina,-0.589938,-0.243718,0.059398
0,North Carolina,-1.577882,-0.236305,0.055840
0,Alabama,-0.167115,-0.178455,0.031846
0,Louisiana,-1.113480,-0.099991,0.009998
0,Tennessee,-0.196256,-0.087673,0.007687
0,Oregon,-0.233879,-0.071688,0.005139
0,Oklahoma,-0.133396,-0.051612,0.002664
0,Illinois,-0.094607,-0.049149,0.002416


---
### Section 2: By incidence rate

$\delta_d = t_2 - t_1$

* $\delta_d$ is the duration (days) between two time points.
* $t$ is the date when the local incidence rate reaches a certain threshold.

**Incidence rate or mortality rate**

In [11]:
# calculate incidence rate
# ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.div.html#pandas.DataFrame.div
temp_df = cases_df.iloc[:,5:]
incidence_df = temp_df.div(cases_df['Population'], axis='rows')*100000
# ref: https://discuss.codecademy.com/t/can-we-add-a-new-column-at-a-specific-position-in-a-pandas-dataframe/355842
incidence_df.insert(0, 'FIPS', cases_df['FIPS'])
incidence_df.insert(1, 'County', cases_df['County'])
incidence_df.insert(2, 'Province_State', cases_df['Province_State'])
incidence_df.insert(3, 'MMR', cases_df['MMR'])
incidence_df.insert(4, 'Population', cases_df['Population'])
print(len(incidence_df))
incidence_df.tail(3)

2674


,FIPS,County,Province_State,MMR,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20
2671,56043,Washakie,Wyoming,0.94,8064,0.0,0.0,0.0,0.0,0.0,...,954.861111,954.861111,979.662698,1016.865079,1066.468254,1190.476190,1202.876984,1202.876984,1240.079365,1264.880952
2672,56045,Weston,Wyoming,0.92,6927,0.0,0.0,0.0,0.0,0.0,...,72.181319,86.617583,72.181319,72.181319,72.181319,72.181319,101.053847,101.053847,115.490111,115.490111
2673,36061,NYC,New York,0.99,8622698,0.0,0.0,0.0,0.0,0.0,...,2642.235644,2644.984203,2648.161863,2652.638420,2657.718037,2661.974245,2666.404413,2669.964784,2672.690149,2675.983781


In [12]:
# result_df = incidence_df
# output_csv = output_folder + 'output_incidence_rate_county_' + t.strftime('%m%d%y%H%M') + '.csv'
# result_df.to_csv(output_csv, index=False, encoding='utf-8')

### --- CHANGE BELOW ---

In [13]:
if cases_or_deaths == 'ratio':
    incidence_df = cases_df

### --- CHANGE ABOVE ---

In [29]:
fig = px.histogram(incidence_df, x="3/10/20", width=600, height=400)
fig.show()

In [30]:
fig = px.histogram(incidence_df, x="4/10/20", width=600, height=400)
fig.show()

In [15]:
fig = px.histogram(incidence_df, x=cases_df.columns[-1], width=600, height=400)
print("Date: " + cases_df.columns[-1])
fig.show()

Date: 8/18/20


### Find the date for a given incidence rate

* Input: incidence value
* Output: FIPS, date, incidence

In [16]:
# case-fatality ratio
# inc_value1 = 0
# inc_value2 = 0.05

# mortality rate
# inc_value1 = 1
# inc_value2 = 30

# incidence rate
inc_value1 = 50
inc_value2 = 1500

In [17]:
# reframe the incidence dataframe
inc_df = incidence_df.drop(columns=['County','Province_State','MMR','Population'])
inc_df = inc_df.set_index('FIPS')
inc_df = inc_df.stack().reset_index()
inc_df = inc_df.rename(columns={"level_1": "date",0: "incidence"})
print(len(inc_df))
inc_df.head()

553350


,FIPS,date,incidence
0,1001,1/22/20,0.0
1,1001,1/23/20,0.0
2,1001,1/24/20,0.0
3,1001,1/25/20,0.0
4,1001,1/26/20,0.0


In [18]:
# extract the first date on or after the incidence threshold
# ref: https://stackoverflow.com/questions/20067636/pandas-dataframe-get-first-row-of-each-group
inc_df1 = inc_df[inc_df['incidence']>=inc_value1]
inc_df1 = inc_df1.groupby('FIPS').first().reset_index()
inc_df2 = inc_df[inc_df['incidence']>=inc_value2]
inc_df2 = inc_df2.groupby('FIPS').first().reset_index()

In [19]:
print(len(inc_df1))
inc_df1.head()

2629


,FIPS,date,incidence
0,1001,4/20/20,50.446815
1,1003,4/18/20,51.263239
2,1005,4/16/20,55.401662
3,1007,4/11/20,57.349568
4,1009,4/23/20,53.436299


In [20]:
print(len(inc_df2))
inc_df2.head()

768


,FIPS,date,incidence
0,1001,7/20/20,1509.801095
1,1003,8/3/20,1509.678876
2,1005,7/11/20,1559.161061
3,1007,7/30/20,1517.557791
4,1009,8/18/20,1528.967645


In [21]:
inc_df3 = pd.merge(inc_df2,inc_df1, how='left', on='FIPS')
print(len(inc_df3))
inc_df3.head(3)

768


,FIPS,date_x,incidence_x,date_y,incidence_y
0,1001,7/20/20,1509.801095,4/20/20,50.446815
1,1003,8/3/20,1509.678876,4/18/20,51.263239
2,1005,7/11/20,1559.161061,4/16/20,55.401662


In [22]:
# counties without MMR
inc_df3[inc_df3['FIPS'].isin(set(inc_df3['FIPS']) - set(mmr_df['FIPS']))]

,FIPS,date_x,incidence_x,date_y,incidence_y


In [23]:
inc_mmr_df = pd.merge(inc_df3,mmr_df, how='left', on='FIPS')
print(len(inc_mmr_df))
inc_mmr_df.head(3)

768


,FIPS,date_x,incidence_x,date_y,incidence_y,County,State,MMR,Population
0,1001,7/20/20,1509.801095,4/20/20,50.446815,Autauga,AL,0.9536,55504
1,1003,8/3/20,1509.678876,4/18/20,51.263239,Baldwin,AL,0.9700,212628
2,1005,7/11/20,1559.161061,4/16/20,55.401662,Barbour,AL,0.9283,25270


In [24]:
# calcuate the duration between date_x and date_y
# ref: https://stackoverflow.com/questions/22132525/add-column-with-number-of-days-between-dates-in-dataframe-pandas
inc_mmr_df['date_x'] = pd.to_datetime(inc_mmr_df['date_x'])
inc_mmr_df['date_y'] = pd.to_datetime(inc_mmr_df['date_y'])
inc_mmr_df['delta_d'] = (inc_mmr_df['date_x'] - inc_mmr_df['date_y']).dt.days
inc_mmr_df.head()

,FIPS,date_x,incidence_x,date_y,incidence_y,County,State,MMR,Population,delta_d
0,1001,2020-07-20,1509.801095,2020-04-20,50.446815,Autauga,AL,0.9536,55504,91
1,1003,2020-08-03,1509.678876,2020-04-18,51.263239,Baldwin,AL,0.9700,212628,107
2,1005,2020-07-11,1559.161061,2020-04-16,55.401662,Barbour,AL,0.9283,25270,86
3,1007,2020-07-30,1517.557791,2020-04-11,57.349568,Bibb,AL,0.9432,22668,110
4,1009,2020-08-18,1528.967645,2020-04-23,53.436299,Blount,AL,0.9740,58013,117


In [25]:
# calcuate the duration between stay_at_home_date and SHO_expired_p1
# ref: https://stackoverflow.com/questions/22132525/add-column-with-number-of-days-between-dates-in-dataframe-pandas
inc_mmr_df['date_x'] = pd.to_datetime(inc_mmr_df['date_x'])
inc_mmr_df['date_y'] = pd.to_datetime(inc_mmr_df['date_y'])
inc_mmr_df['delta_d'] = (inc_mmr_df['date_x'] - inc_mmr_df['date_y']).dt.days
print(len(inc_mmr_df))
inc_mmr_df.head()

768


,FIPS,date_x,incidence_x,date_y,incidence_y,County,State,MMR,Population,delta_d
0,1001,2020-07-20,1509.801095,2020-04-20,50.446815,Autauga,AL,0.9536,55504,91
1,1003,2020-08-03,1509.678876,2020-04-18,51.263239,Baldwin,AL,0.9700,212628,107
2,1005,2020-07-11,1559.161061,2020-04-16,55.401662,Barbour,AL,0.9283,25270,86
3,1007,2020-07-30,1517.557791,2020-04-11,57.349568,Bibb,AL,0.9432,22668,110
4,1009,2020-08-18,1528.967645,2020-04-23,53.436299,Blount,AL,0.9740,58013,117


In [26]:
fig = px.histogram(inc_mmr_df, x="delta_d", width=600, height=400)
fig.show()

In [27]:
# plot it

df = inc_mmr_df

fig1 = px.scatter(df, x="MMR", y="delta_d", trendline="ols") #, text="County"
fig1.update_traces(textposition='top center')
fig1.update_layout(
        title= 'delta vs. MMR',
        xaxis=dict(
            title='MMR',
            gridcolor='white',
            gridwidth=2,
        ),
        yaxis=dict(
            title="Days",
            gridcolor='white',
            gridwidth=2,
        ),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)', width=600, height=400
    )
fig1.show()

In [28]:
state_list = list(set(inc_mmr_df['State']))
state_list.sort(reverse=False)

# from numpy import cov
# from scipy.stats import pearsonr

df = inc_mmr_df
corr_df = pd.DataFrame()

for state in state_list:
    
    df_state = df[(df['State']==state)]
    
    if len(df_state) >= 4:
    
        # covariance covariance[0,1],
        data1 = df_state['MMR']
        data2 = df_state['delta_d']
        covariance = cov(data1, data2)

        # Pearsons correlation
        corr, _ = pearsonr(data1, data2)

        # R-squared
        correlation_matrix = np.corrcoef(data1, data2)
        correlation_xy = correlation_matrix[0,1]
        r_squared = correlation_xy**2

        # outputs
        corr_df = corr_df.append([[state,str(len(df_state)),str(len(mmr_df[mmr_df['State'] == state])),corr,'%.5f' % r_squared]])

corr_df.columns=['state','tested_counties', 'total_counties','pearsons','r_squared']
corr_df.sort_values(by='pearsons', ascending=True).reset_index(drop=True)

,state,tested_counties,total_counties,pearsons,r_squared
0,WI,4,72,-0.974580,0.94981
1,IN,8,92,-0.667288,0.44527
2,CO,8,63,-0.387276,0.14998
3,KS,14,105,-0.384663,0.14797
4,FL,60,67,-0.362519,0.13142
5,AZ,12,15,-0.361720,0.13084
6,OK,6,77,-0.308907,0.09542
7,MO,10,115,-0.290334,0.08429
8,TN,44,95,-0.249790,0.06240
9,KY,8,120,-0.216588,0.04691


---
### Section 3: Days from the i-th cases to the peak

$\delta_d = d_p - d_s$

* $d_p$ is the date with the highest daily cases in 7-day moving average for each county.
* $d_s$ is the date with the 100th cases for each county.
* $delta_d$ is the duration (days) from $d_s$ to  $d_p$.

**cases or deaths**

In [238]:
# case-fatality ratio
#case_value1 = 0

# deaths
# case_value1 = 3

# cases
case_value1 = 10

In [239]:
# base dataframe
cases_df.head()

,FIPS,County,MMR,Population,Province_State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20
0,1001,Autauga,0.9536,55504,Alabama,0.0,0.0,0.0,0.0,0.0,...,1086.0,1169.0,1174.0,1162.0,1188.0,1196.0,1196.0,1206.0,1218.0,1219.0
1,1003,Baldwin,0.9700,212628,Alabama,0.0,0.0,0.0,0.0,0.0,...,3502.0,3544.0,3651.0,3670.0,3710.0,3744.0,3791.0,3839.0,3877.0,3888.0
2,1005,Barbour,0.9283,25270,Alabama,0.0,0.0,0.0,0.0,0.0,...,573.0,575.0,576.0,579.0,581.0,586.0,590.0,598.0,604.0,605.0
3,1007,Bibb,0.9432,22668,Alabama,0.0,0.0,0.0,0.0,0.0,...,426.0,438.0,442.0,444.0,453.0,457.0,465.0,469.0,469.0,474.0
4,1009,Blount,0.9740,58013,Alabama,0.0,0.0,0.0,0.0,0.0,...,788.0,800.0,812.0,816.0,825.0,835.0,848.0,854.0,860.0,863.0


In [240]:
# reframe the case dataframe
daily_df = cases_df.drop(columns=['County','Province_State','MMR','Population'])
daily_df = daily_df.set_index('FIPS')
daily_df = daily_df.stack().reset_index()
daily_df = daily_df.rename(columns={"level_1": "date",0: "cases"})
print(len(daily_df))
daily_df.head()

550715


,FIPS,date,cases
0,1001,1/22/20,0.0
1,1001,1/23/20,0.0
2,1001,1/24/20,0.0
3,1001,1/25/20,0.0
4,1001,1/26/20,0.0


In [241]:
# calculate the daily cases
# ref: https://stackoverflow.com/questions/48347497/pandas-groupby-diff
daily_df['daily'] = daily_df.groupby(['FIPS'])['cases'].diff().fillna(0)
daily_df.tail()

,FIPS,date,cases,daily
550710,36061,8/13/20,229167.0,438.0
550711,36061,8/14/20,229534.0,367.0
550712,36061,8/15/20,229916.0,382.0
550713,36061,8/16/20,230223.0,307.0
550714,36061,8/17/20,230458.0,235.0


In [242]:
# calculate the moving average
# ref: https://stackoverflow.com/questions/53339021/python-pandas-calculate-moving-average-within-group
daily_df['moving_7'] = daily_df.groupby('FIPS')['daily'].transform(lambda x: x.rolling(7, 1).mean())
daily_df.tail()

,FIPS,date,cases,daily,moving_7
550710,36061,8/13/20,229167.0,438.0,321.857143
550711,36061,8/14/20,229534.0,367.0,325.142857
550712,36061,8/15/20,229916.0,382.0,333.142857
550713,36061,8/16/20,230223.0,307.0,341.571429
550714,36061,8/17/20,230458.0,235.0,341.285714


In [243]:
# result_df = df
# output_csv = output_folder + 'output_daily_cases_moving_' + t.strftime('%m%d%y%H%M') + '.csv'
# result_df.to_csv(output_csv, index=False, encoding='utf-8')

In [244]:
# first day on or beyond the threshold
daily_df1 = daily_df[daily_df['moving_7']>=case_value1]
daily_df1 = daily_df1.groupby('FIPS').first().reset_index()
daily_df1

,FIPS,date,cases,daily,moving_7
0,1001,6/11/20,312.0,17.0,10.142857
1,1003,6/24/20,449.0,14.0,10.285714
2,1005,7/16/20,444.0,18.0,11.000000
3,1007,8/9/20,438.0,12.0,10.571429
4,1009,7/12/20,320.0,24.0,12.142857
...,...,...,...,...,...
1140,55133,3/31/20,101.0,8.0,10.000000
1141,55135,7/26/20,312.0,37.0,13.857143
1142,55139,5/30/20,238.0,14.0,10.285714
1143,55141,8/2/20,238.0,9.0,10.428571


In [245]:
# find the peak
daily_df2 = daily_df.sort_values(["FIPS", "moving_7"], ascending = (True,False))
daily_df2 = daily_df2.groupby('FIPS').first().reset_index()
daily_df2

,FIPS,date,cases,daily,moving_7
0,1001,8/9/20,1169.0,83.0,22.714286
1,1003,7/23/20,2423.0,259.0,120.571429
2,1005,7/16/20,444.0,18.0,11.000000
3,1007,8/9/20,438.0,12.0,10.571429
4,1009,7/25/20,569.0,17.0,21.142857
...,...,...,...,...,...
2630,56037,7/23/20,206.0,5.0,6.857143
2631,56039,7/28/20,311.0,19.0,12.142857
2632,56041,6/19/20,128.0,12.0,9.428571
2633,56043,8/9/20,77.0,7.0,4.000000


In [246]:
daily_df3 = pd.merge(daily_df2,daily_df1, how='left', on='FIPS')
print(len(daily_df3))
daily_df3.head(3)

2635


,FIPS,date_x,cases_x,daily_x,moving_7_x,date_y,cases_y,daily_y,moving_7_y
0,1001,8/9/20,1169.0,83.0,22.714286,6/11/20,312.0,17.0,10.142857
1,1003,7/23/20,2423.0,259.0,120.571429,6/24/20,449.0,14.0,10.285714
2,1005,7/16/20,444.0,18.0,11.000000,7/16/20,444.0,18.0,11.000000


In [247]:
# counties without MMR
daily_df3[daily_df3['FIPS'].isin(set(daily_df3['FIPS']) - set(mmr_df['FIPS']))]

,FIPS,date_x,cases_x,daily_x,moving_7_x,date_y,cases_y,daily_y,moving_7_y


In [248]:
daily_mmr_df = pd.merge(daily_df3,mmr_df, how='left', on='FIPS')
print(len(daily_mmr_df))
daily_mmr_df.head(3)

2635


,FIPS,date_x,cases_x,daily_x,moving_7_x,date_y,cases_y,daily_y,moving_7_y,County,State,MMR,Population
0,1001,8/9/20,1169.0,83.0,22.714286,6/11/20,312.0,17.0,10.142857,Autauga,AL,0.9536,55504
1,1003,7/23/20,2423.0,259.0,120.571429,6/24/20,449.0,14.0,10.285714,Baldwin,AL,0.9700,212628
2,1005,7/16/20,444.0,18.0,11.000000,7/16/20,444.0,18.0,11.000000,Barbour,AL,0.9283,25270


In [249]:
# calcuate the duration between date_x and date_y
# ref: https://stackoverflow.com/questions/22132525/add-column-with-number-of-days-between-dates-in-dataframe-pandas
daily_mmr_df['date_x'] = pd.to_datetime(daily_mmr_df['date_x'])
daily_mmr_df['date_y'] = pd.to_datetime(daily_mmr_df['date_y'])
daily_mmr_df['delta_d'] = (daily_mmr_df['date_x'] - daily_mmr_df['date_y']).dt.days
print(len(daily_mmr_df))
daily_mmr_df.head()

2635


,FIPS,date_x,cases_x,daily_x,moving_7_x,date_y,cases_y,daily_y,moving_7_y,County,State,MMR,Population,delta_d
0,1001,2020-08-09,1169.0,83.0,22.714286,2020-06-11,312.0,17.0,10.142857,Autauga,AL,0.9536,55504,59.0
1,1003,2020-07-23,2423.0,259.0,120.571429,2020-06-24,449.0,14.0,10.285714,Baldwin,AL,0.9700,212628,29.0
2,1005,2020-07-16,444.0,18.0,11.000000,2020-07-16,444.0,18.0,11.000000,Barbour,AL,0.9283,25270,0.0
3,1007,2020-08-09,438.0,12.0,10.571429,2020-08-09,438.0,12.0,10.571429,Bibb,AL,0.9432,22668,0.0
4,1009,2020-07-25,569.0,17.0,21.142857,2020-07-12,320.0,24.0,12.142857,Blount,AL,0.9740,58013,13.0


In [250]:
daily_mmr_df = daily_mmr_df[daily_mmr_df['delta_d'].notnull()]
daily_mmr_df = daily_mmr_df[daily_mmr_df['delta_d']>0]
print(len(daily_mmr_df))
daily_mmr_df

1071


,FIPS,date_x,cases_x,daily_x,moving_7_x,date_y,cases_y,daily_y,moving_7_y,County,State,MMR,Population,delta_d
0,1001,2020-08-09,1169.0,83.0,22.714286,2020-06-11,312.0,17.0,10.142857,Autauga,AL,0.9536,55504,59.0
1,1003,2020-07-23,2423.0,259.0,120.571429,2020-06-24,449.0,14.0,10.285714,Baldwin,AL,0.9700,212628,29.0
4,1009,2020-07-25,569.0,17.0,21.142857,2020-07-12,320.0,24.0,12.142857,Blount,AL,0.9740,58013,13.0
5,1011,2020-05-28,183.0,7.0,16.857143,2020-05-23,105.0,16.0,10.000000,Bullock,AL,0.8777,10309,5.0
6,1013,2020-05-12,224.0,28.0,14.857143,2020-05-02,92.0,27.0,10.142857,Butler,AL,0.9729,19825,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,55133,2020-07-27,3092.0,91.0,124.428571,2020-03-31,101.0,8.0,10.000000,Waukesha,WI,0.8300,400621,118.0
2608,55135,2020-07-27,330.0,18.0,15.142857,2020-07-26,312.0,37.0,13.857143,Waupaca,WI,0.8700,51225,1.0
2610,55139,2020-06-10,457.0,26.0,24.714286,2020-05-30,238.0,14.0,10.285714,Winnebago,WI,0.8700,170414,11.0
2611,55141,2020-08-16,379.0,7.0,12.571429,2020-08-02,238.0,9.0,10.428571,Wood,WI,0.8700,73126,14.0


In [251]:
fig = px.histogram(daily_mmr_df, x="delta_d", width=600, height=400)
fig.show()

In [252]:
# plot it

df = daily_mmr_df

fig1 = px.scatter(df, x="MMR", y="delta_d", trendline="ols") #, text="County"
fig1.update_traces(textposition='top center')
fig1.update_layout(
        title= 'delta vs. MMR',
        xaxis=dict(
            title='MMR',
            gridcolor='white',
            gridwidth=2,
        ),
        yaxis=dict(
            title="Days",
            gridcolor='white',
            gridwidth=2,
        ),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)', width=600, height=400
    )
fig1.show()

In [253]:
state_list = list(set(daily_mmr_df['State']))
state_list.sort(reverse=False)

df = daily_mmr_df
corr_df = pd.DataFrame()

for state in state_list:
    
    df_state = df[(df['State']==state)]
    
    if len(df_state) >= 4:
    
        # covariance covariance[0,1],
        data1 = df_state['MMR']
        data2 = df_state['delta_d']
        covariance = cov(data1, data2)

        # Pearsons correlation
        corr, _ = pearsonr(data1, data2)

        # R-squared
        correlation_matrix = np.corrcoef(data1, data2)
        correlation_xy = correlation_matrix[0,1]
        r_squared = correlation_xy**2

        # outputs
        corr_df = corr_df.append([[state,str(len(df_state)),str(len(mmr_df[mmr_df['State'] == state])),corr,'%.5f' % r_squared]])

corr_df.columns=['state','tested_counties', 'total_counties','pearsons','r_squared']
corr_df.sort_values(by='pearsons', ascending=True).reset_index(drop=True)

,state,tested_counties,total_counties,pearsons,r_squared
0,MT,4,56,-0.951479,0.90531
1,CT,8,8,-0.751528,0.56479
2,TN,63,95,-0.487978,0.23812
3,WI,25,72,-0.455878,0.20782
4,FL,67,67,-0.424470,0.18017
5,IL,32,102,-0.414435,0.17176
6,CO,14,63,-0.355539,0.12641
7,VA,44,133,-0.348145,0.12121
8,AZ,14,15,-0.298762,0.08926
9,MO,31,115,-0.280722,0.07880


---
### Section 4: Cases from the stay-at-home order to the peak

In [282]:
time1 = 'public_schools' # 'stay_at_home_date' # 'public_schools'
time2 = 'SHO_expired_p1'

#### --- DO NOT MODIFY BELOW ---

In [283]:
# import policy time
in_table = folder + r'policy_time_US.xlsx'
policy_df = pd.read_excel(in_table,sheet_name='NYT')
# define time points
#time1 = 'public_schools' # 'stay_at_home_date' #
#time2 = 'SHO_expired_p1'
policy_df = policy_df[['Province_State',time1,time2]]
# remove nulls
policy_df = policy_df[policy_df[time1].notnull()]
policy_df = policy_df[policy_df[time2].notnull()]
# date format update
policy_df[time1] = pd.to_datetime(policy_df[time1]).dt.strftime('%#m/%#d/%y')
policy_df[time2] = pd.to_datetime(policy_df[time2]).dt.strftime('%#m/%#d/%y')
# remove rows
policy_df = policy_df[~policy_df['Province_State'].isin(ex_list)]
print(len(policy_df))
policy_df.head(3)

43


,Province_State,public_schools,SHO_expired_p1
0,Alabama,3/26/20,4/30/20
2,Arizona,3/16/20,5/15/20
4,California,3/19/20,5/8/20


In [284]:
state_list = list(set(policy_df['Province_State']))
state_list.sort(reverse=False)

In [285]:
# states without t1 or t2
set(cases_df['Province_State']) - set(state_list)

{'North Dakota', 'South Dakota', 'Utah', 'Vermont', 'Wyoming', nan}

In [286]:
# states without MMR
set(state_list) - set(cases_df['Province_State'])

{'Delaware',
 'District of Columbia',
 'Hawaii',
 'Mississippi',
 'Nevada',
 'New Hampshire',
 'Ohio',
 'West Virginia'}

In [287]:
state_df = pd.DataFrame()
for state in state_list:
    time_point1 = policy_df[policy_df['Province_State']==state][time1].iloc[0]
    time_point2 = policy_df[policy_df['Province_State']==state][time2].iloc[0]
    state_df_temp = cases_df[cases_df['Province_State']==state]
    state_df_temp.insert(5, 't1_date', time_point1)
    state_df_temp.insert(6, 't2_date', time_point2)
    state_df_temp = state_df_temp[['FIPS','County','Province_State','MMR','Population','t1_date','t2_date',time_point1,time_point2]]
    state_df_temp = state_df_temp.rename(columns={time_point1: "t1",time_point2: "t2"})
    state_df = state_df.append(state_df_temp, ignore_index=True)
print(str(len(state_df)) + ' counties, ' + str(len(set(state_df['Province_State']))) + ' states.')
state_df.head(3)

2472 counties, 35 states.


,FIPS,County,Province_State,MMR,Population,t1_date,t2_date,t1,t2
0,1001,Autauga,Alabama,0.9536,55504,3/26/20,4/30/20,6.0,44.0
1,1003,Baldwin,Alabama,0.9700,212628,3/26/20,4/30/20,5.0,174.0
2,1005,Barbour,Alabama,0.9283,25270,3/26/20,4/30/20,0.0,39.0


In [288]:
# calculate delta_c
state_df.loc[state_df['t1']==0, 't1'] = 1
state_df['delta_c'] = (state_df['t2'] - state_df['t1']).div(state_df['t1'])
state_df = state_df[state_df['delta_c'] > 0]
print(len(state_df))
state_df.head()

2178


,FIPS,County,Province_State,MMR,Population,t1_date,t2_date,t1,t2,delta_c
0,1001,Autauga,Alabama,0.9536,55504,3/26/20,4/30/20,6.0,44.0,6.333333
1,1003,Baldwin,Alabama,0.9700,212628,3/26/20,4/30/20,5.0,174.0,33.800000
2,1005,Barbour,Alabama,0.9283,25270,3/26/20,4/30/20,1.0,39.0,38.000000
3,1007,Bibb,Alabama,0.9432,22668,3/26/20,4/30/20,1.0,42.0,41.000000
4,1009,Blount,Alabama,0.9740,58013,3/26/20,4/30/20,2.0,37.0,17.500000


#### --- DO NOT MODIFY ABOVE ---

In [289]:
# stay-at-home order date
sah_df = state_df.drop(columns=['t2_date','t2','delta_c'])
print(len(sah_df))
sah_df.head()

2178


,FIPS,County,Province_State,MMR,Population,t1_date,t1
0,1001,Autauga,Alabama,0.9536,55504,3/26/20,6.0
1,1003,Baldwin,Alabama,0.9700,212628,3/26/20,5.0
2,1005,Barbour,Alabama,0.9283,25270,3/26/20,1.0
3,1007,Bibb,Alabama,0.9432,22668,3/26/20,1.0
4,1009,Blount,Alabama,0.9740,58013,3/26/20,2.0


In [290]:
daily_df2 = daily_df2[daily_df2['moving_7']<50]
fig = px.histogram(daily_df2, x="moving_7", width=600, height=400)
fig.show()

In [291]:
# peak date
print(len(daily_df2))
daily_df2.head()

2269


,FIPS,date,cases,daily,moving_7
0,1001,8/9/20,1169.0,83.0,22.714286
2,1005,7/16/20,444.0,18.0,11.000000
3,1007,8/9/20,438.0,12.0,10.571429
4,1009,7/25/20,569.0,17.0,21.142857
5,1011,5/28/20,183.0,7.0,16.857143


In [292]:
# merge and calculate the duration
merge_df1 = pd.merge(sah_df, daily_df2, how='left', on='FIPS')
print(len(merge_df1))
merge_df1.head(3)

2178


,FIPS,County,Province_State,MMR,Population,t1_date,t1,date,cases,daily,moving_7
0,1001,Autauga,Alabama,0.9536,55504,3/26/20,6.0,8/9/20,1169.0,83.0,22.714286
1,1003,Baldwin,Alabama,0.9700,212628,3/26/20,5.0,NaN,NaN,NaN,NaN
2,1005,Barbour,Alabama,0.9283,25270,3/26/20,1.0,7/16/20,444.0,18.0,11.000000


In [293]:
# calcuate the duration between date_x and date_y
# ref: https://stackoverflow.com/questions/22132525/add-column-with-number-of-days-between-dates-in-dataframe-pandas
merge_df1['t1_date'] = pd.to_datetime(merge_df1['t1_date'])
merge_df1['date'] = pd.to_datetime(merge_df1['date'])
merge_df1['delta_c'] = (merge_df1['date'] - merge_df1['t1_date']).dt.days
print(len(merge_df1))
merge_df1.head()

2178


,FIPS,County,Province_State,MMR,Population,t1_date,t1,date,cases,daily,moving_7,delta_c
0,1001,Autauga,Alabama,0.9536,55504,2020-03-26,6.0,2020-08-09,1169.0,83.0,22.714286,136.0
1,1003,Baldwin,Alabama,0.9700,212628,2020-03-26,5.0,NaT,NaN,NaN,NaN,NaN
2,1005,Barbour,Alabama,0.9283,25270,2020-03-26,1.0,2020-07-16,444.0,18.0,11.000000,112.0
3,1007,Bibb,Alabama,0.9432,22668,2020-03-26,1.0,2020-08-09,438.0,12.0,10.571429,136.0
4,1009,Blount,Alabama,0.9740,58013,2020-03-26,2.0,2020-07-25,569.0,17.0,21.142857,121.0


In [294]:
merge_df1 = merge_df1[merge_df1['delta_c'].notnull()]
merge_df1 = merge_df1[merge_df1['delta_c']>0]
print(len(merge_df1))
merge_df1.head()

1818


,FIPS,County,Province_State,MMR,Population,t1_date,t1,date,cases,daily,moving_7,delta_c
0,1001,Autauga,Alabama,0.9536,55504,2020-03-26,6.0,2020-08-09,1169.0,83.0,22.714286,136.0
2,1005,Barbour,Alabama,0.9283,25270,2020-03-26,1.0,2020-07-16,444.0,18.0,11.000000,112.0
3,1007,Bibb,Alabama,0.9432,22668,2020-03-26,1.0,2020-08-09,438.0,12.0,10.571429,136.0
4,1009,Blount,Alabama,0.9740,58013,2020-03-26,2.0,2020-07-25,569.0,17.0,21.142857,121.0
5,1011,Bullock,Alabama,0.8777,10309,2020-03-26,2.0,2020-05-28,183.0,7.0,16.857143,63.0


In [295]:
fig = px.histogram(merge_df1, x="delta_c", width=600, height=400)
fig.show()

In [296]:
# plot it

df = merge_df1

fig1 = px.scatter(df, x="MMR", y="delta_c", trendline="ols") #, text="County"
fig1.update_traces(textposition='top center')
fig1.update_layout(
        title= 'delta vs. MMR',
        xaxis=dict(
            title='MMR',
            gridcolor='white',
            gridwidth=2,
        ),
        yaxis=dict(
            title="Cases",
            gridcolor='white',
            gridwidth=2,
        ),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)', width=600, height=400
    )
fig1.show()

In [297]:
state_list = list(set(merge_df1['Province_State']))
state_list.sort(reverse=False)

In [298]:
df = merge_df1
corr_df = pd.DataFrame()

for state in state_list:
    
    df_state = df[(df['Province_State']==state)]
    
    if len(df_state) >= 4:
    
        # covariance covariance[0,1],
        data1 = df_state['MMR']
        data2 = df_state['delta_c']
        covariance = cov(data1, data2)

        # Pearsons correlation
        corr, _ = pearsonr(data1, data2)

        # R-squared
        correlation_matrix = np.corrcoef(data1, data2)
        correlation_xy = correlation_matrix[0,1]
        r_squared = correlation_xy**2

        # outputs
        corr_df = corr_df.append([[state,str(len(df_state)),str(len(cases_df[cases_df['Province_State'] == state])),corr,'%.5f' % r_squared]])

corr_df.columns=['state','tested_counties', 'total_counties','pearsons','r_squared']
corr_df.sort_values(by='pearsons', ascending=True).reset_index(drop=True)

,state,tested_counties,total_counties,pearsons,r_squared
0,Connecticut,5,8,-0.871300,0.75916
1,Rhode Island,4,5,-0.625517,0.39127
2,Maryland,19,24,-0.267496,0.07155
3,Louisiana,46,64,-0.254889,0.06497
4,Georgia,138,159,-0.175579,0.03083
5,New Jersey,5,21,-0.173554,0.03012
6,California,25,58,-0.171858,0.02954
7,New Mexico,24,32,-0.149340,0.02230
8,Kentucky,98,120,-0.124231,0.01543
9,Texas,139,243,-0.109270,0.01194


---
### Section 5: By mortality rate

$\delta_d = t_2 - t_1$

* $\delta_d$ is the duration (days) between two time points.
* $t$ is the date when the local mortality rate reaches a certain threshold.

Change parameters: 

1. cases_or_deaths = 'cases'
* mortality_rate
* The i-th deaths

---
### Section 6: By case-fatality ratio

$\delta_d = t_2 - t_1$

* $\delta_d$ is the duration (days) between two time points.
* $t$ is the date when the local case-fatality ratio reaches a certain threshold.